## Preprocessing

In [1]:
import pandas as pd 
import numpy as np 
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
from datetime import timedelta

In [2]:
df_train = pd.read_csv('sales_train_evaluation.csv')
df_train.head() #HOBBIES_1_001	HOBBIES_1	HOBBIES	CA_1	CA	0	2011-01-29

,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,...,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1,0,0,1,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,1,0,0,...,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2,0,0,1,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4,1,0,2,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0


In [4]:
df_cal = pd.read_csv('calendar.csv')
df_cal['date'] = pd.to_datetime(df_cal['date'])
df_cal.head()

,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,0,1


In [5]:
df_sell = pd.read_csv('sell_prices.csv')
df_sell.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [6]:
print(min(df_sell['wm_yr_wk']))
print(max(df_sell['wm_yr_wk']))


11101
11621


In [21]:
df_first = pd.merge(df_train, df_sell, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
df_first.head()

KeyError: 'wm_yr_wk'

### Pivot from long format to short

In [7]:
start_date = pd.to_datetime('2011-01-29')

# Melt the DataFrame to long format directly, without adding separate date columns
melted_df = pd.melt(df_train, 
                    id_vars=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                    value_vars=[f'd_{i}' for i in range(1, 1942)], 
                    var_name='day', 
                    value_name='units_sold')

# Calculate the date directly from the 'day' column
# Extract the day index from the 'day' column (removing 'd_') and convert to int
melted_df['date'] = melted_df['day'].apply(lambda x: start_date + timedelta(days=int(x.split('_')[1])-1))

# Drop the 'day' column if it's no longer needed
melted_df.drop(columns=['day'], inplace=True)

### Add calender information

In [8]:
# HOBBIES_1	HOBBIES
merged_df = pd.merge(melted_df, df_cal, on='date', how='left')
merged_df.head()

,item_id,dept_id,cat_id,store_id,state_id,units_sold,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0


### Add price information

In [9]:
# Left join on 'store_id', 'item_id', and 'wm_yr_wk'
final_df = pd.merge(merged_df, df_sell, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

In [10]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 20 columns):
 #   Column        Dtype         
---  ------        -----         
 0   item_id       object        
 1   dept_id       object        
 2   cat_id        object        
 3   store_id      object        
 4   state_id      object        
 5   units_sold    int64         
 6   date          datetime64[ns]
 7   wm_yr_wk      int64         
 8   weekday       object        
 9   wday          int64         
 10  month         int64         
 11  year          int64         
 12  event_name_1  object        
 13  event_type_1  object        
 14  event_name_2  object        
 15  event_type_2  object        
 16  snap_CA       int64         
 17  snap_TX       int64         
 18  snap_WI       int64         
 19  sell_price    float64       
dtypes: datetime64[ns](1), float64(1), int64(8), object(10)
memory usage: 8.8+ GB


## Reduce memory usage

In [11]:
# Convert object columns with limited unique values to 'category'
for col in ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']:
    final_df[col] = final_df[col].astype('category')

# Convert integer columns to the smallest possible integer type
for col in ['units_sold', 'wm_yr_wk', 'wday', 'month', 'year', 'snap_CA', 'snap_TX', 'snap_WI']:
    final_df[col] = pd.to_numeric(final_df[col], downcast='integer')

# Convert float columns to float32 if precision allows
final_df['sell_price'] = final_df['sell_price'].astype('float32')

# Check memory usage after optimization
print(final_df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 20 columns):
 #   Column        Dtype         
---  ------        -----         
 0   item_id       category      
 1   dept_id       category      
 2   cat_id        category      
 3   store_id      category      
 4   state_id      category      
 5   units_sold    int16         
 6   date          datetime64[ns]
 7   wm_yr_wk      int16         
 8   weekday       category      
 9   wday          int8          
 10  month         int8          
 11  year          int16         
 12  event_name_1  category      
 13  event_type_1  category      
 14  event_name_2  category      
 15  event_type_2  category      
 16  snap_CA       int8          
 17  snap_TX       int8          
 18  snap_WI       int8          
 19  sell_price    float32       
dtypes: category(10), datetime64[ns](1), float32(1), int16(3), int8(5)
memory usage: 1.9 GB
None


In [12]:
print(final_df['sell_price'].isna().sum())

12299413


In [13]:
na_sell = final_df.loc[final_df['sell_price'].isna()]
na_sell.head()

,item_id,dept_id,cat_id,store_id,state_id,units_sold,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


In [19]:
print(na_sell['cat_id'].unique())

['HOBBIES', 'HOUSEHOLD', 'FOODS']
Categories (3, object): ['FOODS', 'HOBBIES', 'HOUSEHOLD']


In [20]:
print(final_df['cat_id'].unique())

['HOBBIES', 'HOUSEHOLD', 'FOODS']
Categories (3, object): ['FOODS', 'HOBBIES', 'HOUSEHOLD']


In [16]:
print(na_sell['date'].min())
print(na_sell['date'].max())

2011-01-29 00:00:00
2016-02-12 00:00:00


In [18]:
test1 = df_sell.loc[df_sell['item_id'] == 'HOBBIES_1_001']
print(test1['wm_yr_wk'].min())
print(test1['wm_yr_wk'].max())

11325
11621


### Save to parquet

In [11]:
# Make the final optimized data to parquet
# final_df.to_parquet('sales_train_eval_processed.parquet', compression='snappy', engine='pyarrow')

/Users/tomaltenborg/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
